## 02 - Azure Data Lake Storage Gen2

In this task, we will look at how to create an Azure Data Lake Storage account and upload a file to it.

1. Open the Azure portal.
2. Search for `Storage accounts` and click on it.
3. Click on the `+ Create` button to create a new storage account.
4. Fill in the necessary details to create a new storage account.
5. After the storage account is created, click on the `Containers` tab and create a new container.
6. Upload a file to the container.